In [24]:
import torch
from vae.model import Encoder_model, Decoder_model
import matplotlib.pyplot as plt
import sys
import os
import json

sys.path.append('/home/pcrespo/repos/tabsyn')
from utils_train import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_dir = '/home/pcrespo/repos/tabsyn/data/adult'
info_path = f'{data_dir}/info.json'

with open(info_path, 'r') as f:
    info = json.load(f)

X_num, X_cat, categories, d_numerical = preprocess(data_dir, task_type = info['task_type'])

X_train_num, _ = X_num
X_train_cat, _ = X_cat

X_train_num, X_test_num = X_num
X_train_cat, X_test_cat = X_cat

X_train_num, X_test_num = torch.tensor(X_train_num).float(), torch.tensor(X_test_num).float()
X_train_cat, X_test_cat =  torch.tensor(X_train_cat), torch.tensor(X_test_cat)

# Set the parameters
num_layers = 2              
d_token = 4                 
n_head = 1                  
factor = 32

pre_encoder = Encoder_model(num_layers, d_numerical, categories, d_token, n_head = n_head, factor = factor).to(device)
pre_decoder = Decoder_model(num_layers, d_numerical, categories, d_token, n_head = n_head, factor = factor).to(device)

pre_encoder.load_state_dict(torch.load('/home/pcrespo/repos/tabsyn/tabsyn/vae/ckpt/adult/encoder.pt', map_location=device))
pre_encoder.eval()
pre_decoder.load_state_dict(torch.load('/home/pcrespo/repos/tabsyn/tabsyn/vae/ckpt/adult/decoder.pt', map_location=device))
pre_decoder.eval()


No NaNs in numerical features, skipping
self.category_embeddings.weight.shape=torch.Size([104, 4])


/tmp/ipykernel_3343690/3128044824.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pre_encoder.load_state_dict(torch.load('/home/pcrespo/repos/tabsyn/tabsyn/vae/ckpt/adu

Decoder_model(
  (VAE_Decoder): Transformer(
    (layers): ModuleList(
      (0): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=4, out_features=4, bias=True)
          (W_k): Linear(in_features=4, out_features=4, bias=True)
          (W_v): Linear(in_features=4, out_features=4, bias=True)
        )
        (linear0): Linear(in_features=4, out_features=128, bias=True)
        (linear1): Linear(in_features=128, out_features=4, bias=True)
        (norm1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
      )
      (1): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=4, out_features=4, bias=True)
          (W_k): Linear(in_features=4, out_features=4, bias=True)
          (W_v): Linear(in_features=4, out_features=4, bias=True)
        )
        (linear0): Linear(in_features=4, out_features=128, bias=True)
        (linear1): Linear(in_features=128, out_features=4, bias=True)
        (norm1): LayerNorm((

In [31]:
from tabsyn.vae.model import Model_VAE

model_path = '/home/pcrespo/repos/tabsyn/tabsyn/vae/ckpt/adult/model.pt'

model = Model_VAE(num_layers=num_layers, d_numerical=d_numerical, categories=categories, d_token=d_token, n_head=n_head, factor=factor, bias=True).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

self.category_embeddings.weight.shape=torch.Size([104, 4])


/tmp/ipykernel_3343690/324076232.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Model_VAE(
  (VAE): VAE(
    (Tokenizer): Tokenizer(
      (category_embeddings): Embedding(104, 4)
    )
    (encoder_mu): Transformer(
      (layers): ModuleList(
        (0): ModuleDict(
          (attention): MultiheadAttention(
            (W_q): Linear(in_features=4, out_features=4, bias=True)
            (W_k): Linear(in_features=4, out_features=4, bias=True)
            (W_v): Linear(in_features=4, out_features=4, bias=True)
          )
          (linear0): Linear(in_features=4, out_features=128, bias=True)
          (linear1): Linear(in_features=128, out_features=4, bias=True)
          (norm1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
        )
        (1): ModuleDict(
          (attention): MultiheadAttention(
            (W_q): Linear(in_features=4, out_features=4, bias=True)
            (W_k): Linear(in_features=4, out_features=4, bias=True)
            (W_v): Linear(in_features=4, out_features=4, bias=True)
          )
          (linear0): Linear(in_features=4,

In [34]:
X_test_num = X_test_num.to(device)
X_test_cat = X_test_cat.to(device)

with torch.no_grad():
    Recon_X_num, Recon_X_cat, mu_z, std_z = model(X_test_num, X_test_cat)

In [40]:
Recon_X_num.shape

torch.Size([16281, 6])

In [43]:
Recon_X_cat[0]

tensor([[  2.6523, -30.5911],
        [ -0.8983, -31.0819],
        [  4.6515,  14.0340],
        ...,
        [ -1.9954, -27.8558],
        [  2.6179, -31.5039],
        [  3.7487,  15.5403]], device='cuda:0')